# Dipole Preprocessing
Loading and modifying prerun simulations for dimensionality reduction and/or inference

In [1]:
import os
if 'DISPLAY' in os.environ:
    del os.environ['DISPLAY']
import torch
import dill
import os.path as op
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sbi.utils as utils
from sbi.inference.base import infer
import multiprocessing
import datetime
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from scipy import interpolate
import glob
import pandas as pd

import sys
sys.path.append('../')
import spike_train_functions
import hnn_simnets_functions


/home/ntolley/anaconda3/lib/python3.7/site-packages/elephant/pandas_bridge.py:22: DeprecationWarning: pandas_bridge module will be removed in Elephant v0.8.x
  DeprecationWarning)


In [2]:
save_name ='beta_event_basket_ampa_t100000_02222021_131840'
data_path = '../../data/beta/prerun_simulations/' + save_name + '/'

params_fname_file = open(data_path + 'params_fname_' + save_name + '.pkl', 'rb')
params_fname = dill.load(params_fname_file)
params_fname_file.close()
# params_fname = '../../data/beta/params/beta_param.param'

dpl_files = sorted(glob.glob(data_path + 'data/*dpl*sim*'))
theta_files = sorted(glob.glob(data_path + 'data/*theta*sim*'))
# spike_gids_files = sorted(glob.glob(data_path + 'data/spike_gids*'))
# spike_times_files = sorted(glob.glob(data_path + 'data/spike_times*'))
# spike_types_files = sorted(glob.glob(data_path + 'data/spike_types*'))

In [11]:
# Load downsampled data
downsample = 100
dpl_all = np.vstack([np.loadtxt(dpl_files[file_idx],delimiter=',')[:,::downsample] for file_idx in range(len(dpl_files))])
theta_all = np.vstack([np.loadtxt(theta_files[file_idx],delimiter=',') for file_idx in range(len(theta_files))])

sim_length = 500 # in ms

sim_times_all = np.linspace(0,sim_length, dpl_all.shape[1])

#Filter to subthreshold events
threshold = 100000
threshold_mask  = np.all(np.abs(dpl_all) < threshold, axis=1)
dpl_subthreshold = dpl_all[threshold_mask,:]
theta = theta_all[threshold_mask,:]

#Crop simulation data 
sim_tstart, sim_t_end = 50, 500
sim_times_mask = np.logical_and(sim_times_all > sim_tstart, sim_times_all < sim_t_end)
sim_times = sim_times_all[sim_times_mask]
dpl = dpl_subthreshold[:, sim_times_mask]


In [ ]:
dpl_preprocessed_100x_downsample{
    'dpl': dpl,
    'theta': theta, 
    'sim_times': sim_times,
    'threshold': threshold,
    'subset_included': np.where(threshold_mask)
}

save_file = open(data_path + 'dpl_preprocessed_100x_downsample.pkl', 'wb')
dill.dump(posterior_files, save_file)
save_file.close()